In [2]:
import requests
import hashlib

url = "https://coinmarketcap.com/"

response = requests.get(url)
content = response.text

# Create a hash of the content
content_hash = hashlib.md5(content.encode()).hexdigest()

print(f"Content hash: {content_hash}")



Content hash: 298d0b2ae1c2716360e436a375b58c09


I waited one minute in between these two requests and got a different hash. This is expected as the page constantly updates itself.

In [31]:
url = "https://coinmarketcap.com/"

response = requests.get(url)
content = response.text

# Create a hash of the content
content_hash = hashlib.md5(content.encode()).hexdigest()

print(f"Content hash: {content_hash}")

Content hash: 1bc289d39e2e1425782cf6229523d791


In [30]:
url = "https://time.is/"
response = requests.get(url)
content = response.text

# Create a hash of the content
content_hash = hashlib.md5(content.encode()).hexdigest()

print(f"Content hash: {content_hash}")

Content hash: ec6c267b8d421fbdcddb78b27db44b73


The same experiment for the time.is website fails to update, despite the fact that the page is constantly updating.

In [29]:
url = "https://time.is/"
response = requests.get(url)
content = response.text

# Create a hash of the content
content_hash = hashlib.md5(content.encode()).hexdigest()

print(f"Content hash: {content_hash}")

Content hash: ec6c267b8d421fbdcddb78b27db44b73


In [36]:
import time

url = "https://weather.com/en-GB/weather/hourbyhour/l/82ce477283f4943abb3e6b33ab0958d41e2ae224b2429d8b7decd8fb42e2493f"
response = requests.get(url)
content = response.text
content_hash = hashlib.md5(content.encode()).hexdigest()
print(f"Content hash: {content_hash}")

time.sleep(5)
url_2 = "https://weather.com/en-GB/weather/hourbyhour/l/82ce477283f4943abb3e6b33ab0958d41e2ae224b2429d8b7decd8fb42e2493f"
response_2 = requests.get(url)
content_2 = response_2.text
content_hash_2 = hashlib.md5(content_2.encode()).hexdigest()
print(f"Content_2 hash: {content_hash_2}")


Content hash: 7ea86a5b8c64eb61d45b5bfcea46807f
Content_2 hash: 2a588be8a4a7bf8ca5a362af362fd99e


Conclusion:
- When the content of a page has changed, and required the page to be reloaded, the hash will change.
- When the content of a page has changed, but does not require the page to be reloaded, the hash will not change.

Checking whether the hash has changed is not could be used to determine whether the page has changed.


### Altering the Scrapy Spider 

In [ ]:
import scrapy
